In [1]:
#importing libraries
import pandas as pd
import nltk 
import numpy as np
import string
import nltk
import re 

In [2]:
#Reading the trainig data from txt file
traindata = pd.read_csv('G:\\MSC\\sem3\\Machine learning\\project\\train.txt', sep='\t')

In [3]:
#check thel length of the dataset
len(traindata)

3817

In [4]:
#reading the test data from txt file
testdata = pd.read_csv('G:\\MSC\\sem3\\Machine learning\\project\\test.txt')

In [5]:
#check the test data lenght
len(testdata)

784

# Pre-Processing steps

In [6]:
#combining the 2 datasets for preprocessing 
combi = traindata.append(testdata, ignore_index=True)

C:\Anaconda3\envs\msc_py35\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
#function to remove a given pattern in a string
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [8]:
#Removing mentions in tweets
combi['custom_data'] = np.vectorize(remove_pattern)(combi['Tweet text'], "@[\w]*")

In [9]:
#Removing special characters, numbers, punctuations
combi['custom_data'] = combi['custom_data'].str.replace("[^a-zA-Z#]", " ")

In [10]:
#creating a bag of words
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(combi['custom_data'])

# Building the Model 

In [11]:
#Logistic regression model using BOW as feature 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_bow = bow[:3817,:]
test_bow = bow[3817:,:]

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, data['Label'], random_state=42, test_size=0.3)

lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int) # calculating f1 score

0.7840735068912711

# Predictions

In [12]:
test_pred = lreg.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['Label'] = test_pred_int

In [13]:
test.head()

,Tweet index,Tweet text,Label
0,1,@Callisto1947 Can U Help?||More conservatives ...,0
1,2,"Just walked in to #Starbucks and asked for a ""...",1
2,3,#NOT GONNA WIN http://t.co/Mc9ebqjAqj,0
3,4,@mickymantell He is exactly that sort of perso...,0
4,5,So much #sarcasm at work mate 10/10 #boring 10...,1


In [14]:
submission = test[['Tweet index','Label']]
# writing data to a CSV file
submission.to_csv('sub_lreg_bow.csv', index=False) 